In [2]:
# Import the packages 
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import table
from itertools import permutations 
# import dataframe_image as dfi
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from prettytable import PrettyTable
from tabulate import tabulate
#from tabulate import table

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Import the packages 
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import table
from itertools import permutations 
# import dataframe_image as dfi
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from prettytable import PrettyTable
from tabulate import tabulate
#from tabulate import table


def get_permutations(numbers):
    return list(permutations(numbers))

def convert_to_0s(photon):
    digit_to_0s = [(i-1)*"0" for i in photon]
    return digit_to_0s
    
def AddPolarization(digit_to_0s):
    result = ['']
    prev_non_empty = None
    
    for i, elem in enumerate(digit_to_0s):
        if i == 0:
            prev_non_empty = elem
            result.append(elem)
            continue
        
        if elem.count('0') > prev_non_empty.count('0'):
            result[-1] = 'H' + result[-1]
        elif elem.count('0') < prev_non_empty.count('0'):
            result[-1] = 'V' + result[-1]
        
        result.append(elem)
        prev_non_empty = elem
    
    if len(result[0]) < len(result[-1]):
        result[-1] = 'V' + result[-1][1:] + '0'
    elif len(result[0]) > len(result[-1]):
        result[-1] = 'H' + result[-1][1:] + '0'
    elif len(digit_to_0s) == 2:
        result[-1] = 'H'
    
    if len(digit_to_0s) > 1:
        last_digit_zeros = digit_to_0s[-1].count('0')
        second_last_digit_zeros = digit_to_0s[-2].count('0')
        if last_digit_zeros == 0:
            if second_last_digit_zeros > 0:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] += 'H'
        elif result[-1][-1] == '1':
            if last_digit_zeros > second_last_digit_zeros:
                result[-1] = result[-1][:-1] + 'V'
            elif second_last_digit_zeros > last_digit_zeros:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] = result[-1][:-1] + 'V' if result[-2][-1] == 'H' else result[-1][:-1] + 'H'
    
    return ''.join(result)      # Join the result list into a single string

def convert_to_one_string(numList):         # [1,2,3]
      s = map(str, numList)   # ['1','2','3']
      s = ''.join(s)          # '123'
      s = f'{s}'
      #s = int(s)              # 123
      return s

def flip_polarizations(polarization_string):
    output = {polarization_string}
    for i in range(1, len(polarization_string) + 1): # loop through all possible flip counts
        for j in range(len(polarization_string)): # loop through each polarization
            if polarization_string[j] != '0':
                output |= {polarization_string[:j] + '0' + polarization_string[j+1:]}
                for k in range(j+1, len(polarization_string)):
                    if polarization_string[k] != '0':
                        new_string = polarization_string[:j] + '0' + polarization_string[j+1:k] + '0' + polarization_string[k+1:]
                        output.add(new_string)
        if i == len(polarization_string): # stop flipping if there are no more polarizations to flip
            break
        new_output = set()
        for string in output:
            for j in range(len(polarization_string)):
                if string[j] != '0':
                    for k in range(j+1, len(polarization_string)):
                        if string[k] != '0':
                            new_string = string[:j] + '0' + string[j+1:k] + '0' + string[k+1:]
                            new_output.add(new_string)
        output |= new_output

    output.discard(polarization_string) # remove the original string from the output
    return sorted(output) # return a sorted list of polarization strings.


photons_2 = get_permutations([1,2]) 
photons_3 = get_permutations([1,2,4])
photons_4 = get_permutations([1, 2, 4, 7]) 
photons_5 = get_permutations([1, 2, 4, 7, 12]) 


photons = [photons_2, photons_3, photons_4, photons_5, ]

dictionaries = []

for i, photon_set in enumerate(photons, 2):
    dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
    flipped_list = []

    for j, sub_list in enumerate(photon_set):
        elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
        dictionary["Encoded Words"].append(elements)
        flipped = flip_polarizations(elements)
        flipped_list.append([''.join(f) for f in flipped])
    dictionary[f"Received Loss Errors"] = flipped_list

    dictionaries.append(dictionary)


# for i, dictionary in enumerate(dictionaries):
#     print(f"Dictionary of {i+2} photons: {dictionary}")


# Define table headers
headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# Create table object
table = PrettyTable(headers)

# Set the maximum width for each column
table.max_width["Sent Digits"] = 15
table.max_width["Encoded Words"] = 30
table.max_width["Received Loss Errors"] = 30

# Add rows to table
for d in dictionaries:
    # Get photons and sums values from dictionary
    photons = d["Sent Digits"]
    encoded_words = d["Encoded Words"]
    loss_errors = d["Received Loss Errors"]
    #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
    # Add rows to the table
    for i in range(len(photons)):
        row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
        table.add_row(row)


def check_codeword(codeword, table):
    for row in table._rows:
        sent_digit = row[0]
        encoded_word = row[1]
        received_loss_error = row[2]

        if codeword == received_loss_error:
            print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return

    print("The provided code word is not found in the 'Received Loss Errors' column in the table.")

print(table)


print("Recevied Loss Error")


+-----------------+----------------------------+--------------------------------+
|   Sent Digits   |       Encoded Words        |      Received Loss Errors      |
+-----------------+----------------------------+--------------------------------+
|      (1, 2)     |            HV0             |     ['000', '0V0', 'H00']      |
|      (2, 1)     |            V0H             |     ['000', '00H', 'V00']      |
|    (1, 2, 4)    |          HH0V000           |     ['0000000', '000V000',     |
|                 |                            |     '0H00000', '0H0V000',      |
|                 |                            |     'H000000', 'H00V000',      |
|                 |                            |           'HH00000']           |
|    (1, 4, 2)    |          HV000V0           |     ['0000000', '00000V0',     |
|                 |                            |     '0V00000', '0V000V0',      |
|                 |                            |     'H000000', 'H0000V0',      |
|               

In [ ]:
print("All Received Loss Errors:")
for d in dictionaries:
    loss_errors = d["Received Loss Errors"]
    for sublist in loss_errors:
        for error in sublist:
            print(error)



All Received Loss Errors:
000
0V0
H00
000
00H
V00
0000000
000V000
0H00000
0H0V000
H000000
H00V000
HH00000
0000000
00000V0
0V00000
0V000V0
H000000
H0000V0
HV00000
0000000
000V000
00H0000
00HV000
V000000
V00V000
V0H0000
0000000
000000H
00V0000
00V000H
H000000
H00000H
H0V0000
0000000
00000V0
0000H00
0000HV0
V000000
V0000V0
V000H00
0000000
000000H
0000V00
0000V0H
V000000
V00000H
V000V00
00000000000000
0000000V000000
000H0000000000
000H000V000000
0H000000000000
0H00000V000000
0H0H0000000000
0H0H000V000000
H0000000000000
H000000V000000
H00H0000000000
H00H000V000000
HH000000000000
HH00000V000000
HH0H0000000000
00000000000000
0000000000V000
000V0000000000
000V000000V000
0H000000000000
0H00000000V000
0H0V0000000000
0H0V000000V000
H0000000000000
H000000000V000
H00V0000000000
H00V000000V000
HH000000000000
HH00000000V000
HH0V0000000000
00000000000000
0000000V000000
00000H00000000
00000H0V000000
0V000000000000
0V00000V000000
0V000H00000000
0V000H0V000000
H0000000000000
H000000V000000
H0000H00000000

In [ ]:
def check_codeword(codeword, table):
    if all(c == '0' for c in codeword):
        print("I cannot determine the correct code word. Please check the table.")
        return

    for row_1 in table._rows:
        sent_digit = row_1[0]
        encoded_word = row_1[1]
        received_loss_error = row_1[2]

        if codeword == encoded_word:
            print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return
        
    print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.")
    

def find_matching_row(codeword, table):
    for row_2 in table._rows:
        sent_digits = row_2[0]
        encoded_word = row_2[1]
        received_loss_errors = row_2[2]

        if codeword in received_loss_errors:
            return row_2   
    print("The provided code word is not found in the 'Received Loss Errors' column in the table.")
    return None

In [ ]:
row_1 = check_codeword('V0H', table)
if row_1:
    sent_digits, encoded_word, received_loss_errors = row_1
    print("Matching row found:\nYour received code wrod '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'")

# print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
# return

        # if codeword == row[encoded_word_column]:
        #     sent_digit = row[sent_digit_column]
        #     print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
        #     found = True
        #     break


# Find the matching row for code word '0V0'
row_2 = find_matching_row('V000000000000V000000000000', table)
if row_2:
    sent_digits, encoded_word, received_loss_errors = row_2
    print(f"Matching row found:\nYour received code wrod is in the Received loss error, which belongs to one of the possibilities: '{received_loss_errors}'\nPossible Sent Encoded word: {encoded_word}\nCorresponding Sent digits: {sent_digits}\n")

Your received code word 'V0H' is correct, which corresponds to the sent digit (2, 1) from the dictionary 'Sent Digits'.
Matching row found:
Your received code wrod is in the Received loss error, which belongs to one of the possibilities: '['00000000000000000000000000', '0000000000000000000000000H', '0000000000000V000000000000', '0000000000000V00000000000H', '000000H0000000000000000000', '000000H000000000000000000H', '000000H000000V000000000000', '000000H000000V00000000000H', '0000H000000000000000000000', '0000H00000000000000000000H', '0000H00000000V000000000000', '0000H00000000V00000000000H', '0000H0H0000000000000000000', '0000H0H000000000000000000H', '0000H0H000000V000000000000', '0000H0H000000V00000000000H', 'V0000000000000000000000000', 'V000000000000000000000000H', 'V000000000000V000000000000', 'V000000000000V00000000000H', 'V00000H0000000000000000000', 'V00000H000000000000000000H', 'V00000H000000V000000000000', 'V00000H000000V00000000000H', 'V000H000000000000000000000', 'V000H0000

# Draft

In [ ]:
check_codeword("0V0", table) # I expect it to print(f"Your received code word '{0V0}' is incorrect, it should be the coded word '{HV0}' from the 'Encoded Words', which corresponds to the sent digit {(1,2)} from the dictionary 'Sent Digits'.")
check_codeword("00H", table) # I expect it to print(f"Your received code word '{0HH}' is incorrect, it should be the coded word '{V0H}' from the 'Encoded Words', which corresponds to the sent digit {(2,1)} from the dictionary 'Sent Digits'.")
check_codeword("0V000V0", table) # I expect it to print(f"Your received code word '{0V000V0}' is incorrect, it should be the coded word '{HV000V0}' from the 'Encoded Words', which corresponds to the sent digit {1,4,2)} from the dictionary 'Sent Digits'.")

In [ ]:
# Create a function that flips the polarization of a polarization string and returns a list of all possible polarization strings.
# Take 'HV0' as an example. 
# look how many polarizations there are. 
# Since there are 2 polarizations (H or V) in 'HV0', I expect to have 2! + 1 elements in our list.
# If there is a 0 in the string, we should not flip the 0 to 0.

# flip each corresponding polarization 1 time in each step: 
# we flip the first polarization of the original string to 0, so we have ‘0V0’, and append the output to the list. 
# so we have {'0V0'}.
# we flip the second polarization of the original string to 0 and have ‘H00’ append the output to the list.
# so we have {'0V0', 'H00'}.
# Since there is no third polarization, we finish the loop of flipping each element one time.
# After flipping each element one time, we start a new loop to flip the elements 2 times in each step. 

# flip each corresponding polarization 2 times in each step: 
# flip the first and the second polarization of the original string to 0, so we have '000' and append the output to the list.
# so we have {'0V0', 'H00', '000'}.
# After flipping 2 times, we only have 2 polarizations in the original string so we stop the flipping here. 
# Now we scan the list to see if there are any duplication in the list and original string should not be in the list, we should take them out of the list.
# Look through how many elements in the list, we should have 3 elements in the list, then print the list.

# Take 'H0V000H' as another example.
# look how many polarizations there are. 
# If there are 3 polarizations in the string 'H0V000H', I expect to have 3! + 1 elements in the list and so on.
# If there is a 0 in the string, we should not flip the 0 to 0.

# flip each corresponding polarization 1 time in each step: 
# we flip the first polarization of the original string to 0 append the output to the list. 
# So we have {'00V000H'}. 
# we flip the second polarization of the original string to 0 and append the output to the list.
# So we have {'00V000H', ‘H00000H’} .
# we flip the third polarization of the original string to 0 and append the output to the list. 
# So we have {'00V000H', ‘H00000H’, ‘H0V0000’}.
# Since there is no fourth polarization, we finish the loop of flipping each element one time.
# After flipping each element 1 timein each step, we start a new loop to flip the elements 2 times in each step.

# flip each corresponding polarization 2 times in each step: 
# flip the first and the second polarization of the original string to 0 and append the output to the list
# So we have {'00V000H', ‘H00000H’, ‘H0V0000’, '000000H'}.
# flip the first and the third polarization of the original string to 0 append the output to the list
# So we have {'00V000H', ‘H00000H’, ‘H0V0000’, '000000H','00V0000'}.
# flip the second and the third polarization of the original string to 0  and append the output to the list.  
# So we have {'00V000H', ‘H00000H’, ‘H0V0000’, '000000H','00V0000', 'H000000'}.
# After flipping each element 2 times in each step, we start a new loop to flip the elements 3 times in each step.

# flip each corresponding polarization 3 times in each step: 
# flip the first, the second and the third polarization of the original string to 0, and append the output to the list.
# so we have {'00V000H', ‘H00000H’, ‘H0V0000’, '000000H','00V0000', 'H000000', '0000000'}. 
# After flipping 3 times in each step, we only have 3 polarizations in the original string so we stop the flipping here. 
# Now we scan the list to see if there are any duplication in the list and original string should not be in the list, we should take them out of the list. 
# Look through how many elements in the list, we should have 7 elements in the list, then print the list.

# Therefore, we have a list of all possible polarization strings.
# For     'HV0', I expect to have {‘0V0’, ‘H00’, ‘000’}, no original string in the list, and there should be 3 elements in the list. 
# For     ‘V0H’, I expect to have {'00H', 'V00', '000'},  no original string in the list,  and there should be 3 elements in the list.
# For 'H0V000H', I expect to have ('H0V00H') {'00V000H', 'H00000H', 'H0V0000', '000000H', '00V0000', 'H000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# for 'HV000V0', I expect to have {'0V000V0', 'H0000V0', 'HV00000', '00000V0', '0V00000', 'H000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# for 'V0HV000', I expect to have {'00HV000', 'V00V000', ',V0H0000', '000V000', '00H0000', 'V000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# For 'V000V0H', I expect to have {'0000V0H', 'V00000H', 'V000V00', '000000H', '0000V00', 'V000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# For 'HH0V000', I expect to have {'0H0V000, 'H00V000', 'HH00000', '000V000', '0H00000', 'H0000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# For 'V000HV0', I expect to have {'0000HV0', 'V0000V0', 'V000H00', '00000V0', '0000H00', 'V000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.
# For 'V000V0H', I expect to have {'0000V0H', 'V00000H', 'V000V00', '000000H', '0000V00', 'V000000', '0000000'},  no original string in the list,  and there should be 7 elements in the list.

    


In [ ]:
def flip_polarizations(polarization_string):
    output = {polarization_string}
    for i in range(1, len(polarization_string) + 1): # loop through all possible flip counts
        for j in range(len(polarization_string)): # loop through each polarization
            if polarization_string[j] != '0':
                output |= {polarization_string[:j] + '0' + polarization_string[j+1:]}
                for k in range(j+1, len(polarization_string)):
                    if polarization_string[k] != '0':
                        new_string = polarization_string[:j] + '0' + polarization_string[j+1:k] + '0' + polarization_string[k+1:]
                        output.add(new_string)
        if i == len(polarization_string): # stop flipping if there are no more polarizations to flip
            break
        new_output = set()
        for string in output:
            for j in range(len(polarization_string)):
                if string[j] != '0':
                    for k in range(j+1, len(polarization_string)):
                        if string[k] != '0':
                            new_string = string[:j] + '0' + string[j+1:k] + '0' + string[k+1:]
                            new_output.add(new_string)
        output |= new_output

    output.discard(polarization_string) # remove the original string from the output
    return sorted(output) # return a sorted list of polarization strings.


In [ ]:
flip_polarizations('HV0')

In [ ]:
flip_polarizations('V0H')

In [ ]:
flip_polarizations('HH0V000')

In [ ]:
flip_polarizations('HV000V0')

In [ ]:
flip_polarizations('V0HV000')

In [ ]:
flip_polarizations('H0V000H')

In [ ]:
flip_polarizations('V000HV0')

In [ ]:
flip_polarizations('V000V0H')

In [ ]:
flip_polarizations('HH0H000V000000')

In [ ]:
# pip install matplotlib

In [ ]:
def get_permutations(numbers):
    return list(permutations(numbers))

In [ ]:
def convert_to_0s(photon):
    digit_to_0s = [(i-1)*"0" for i in photon]
    return digit_to_0s
    
def AddPolarization(digit_to_0s):
    result = ['']
    prev_non_empty = None
    
    for i, elem in enumerate(digit_to_0s):
        if i == 0:
            prev_non_empty = elem
            result.append(elem)
            continue
        
        if elem.count('0') > prev_non_empty.count('0'):
            result[-1] = 'H' + result[-1]
        elif elem.count('0') < prev_non_empty.count('0'):
            result[-1] = 'V' + result[-1]
        
        result.append(elem)
        prev_non_empty = elem
    
    if len(result[0]) < len(result[-1]):
        result[-1] = 'V' + result[-1][1:] + '0'
    elif len(result[0]) > len(result[-1]):
        result[-1] = 'H' + result[-1][1:] + '0'
    elif len(digit_to_0s) == 2:
        result[-1] = 'H'
    
    if len(digit_to_0s) > 1:
        last_digit_zeros = digit_to_0s[-1].count('0')
        second_last_digit_zeros = digit_to_0s[-2].count('0')
        if last_digit_zeros == 0:
            if second_last_digit_zeros > 0:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] += 'H'
        elif result[-1][-1] == '1':
            if last_digit_zeros > second_last_digit_zeros:
                result[-1] = result[-1][:-1] + 'V'
            elif second_last_digit_zeros > last_digit_zeros:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] = result[-1][:-1] + 'V' if result[-2][-1] == 'H' else result[-1][:-1] + 'H'
    
    return ''.join(result)      # Join the result list into a single string

def convert_to_one_string(numList):         # [1,2,3]
      s = map(str, numList)   # ['1','2','3']
      s = ''.join(s)          # '123'
      s = f'{s}'
      #s = int(s)              # 123
      return s

In [ ]:
def flip_polarizations(polarization_string):
    output = {polarization_string}
    for i in range(1, len(polarization_string) + 1): # loop through all possible flip counts
        for j in range(len(polarization_string)): # loop through each polarization
            if polarization_string[j] != '0':
                output |= {polarization_string[:j] + '0' + polarization_string[j+1:]}
                for k in range(j+1, len(polarization_string)):
                    if polarization_string[k] != '0':
                        new_string = polarization_string[:j] + '0' + polarization_string[j+1:k] + '0' + polarization_string[k+1:]
                        output.add(new_string)
        if i == len(polarization_string): # stop flipping if there are no more polarizations to flip
            break
        new_output = set()
        for string in output:
            for j in range(len(polarization_string)):
                if string[j] != '0':
                    for k in range(j+1, len(polarization_string)):
                        if string[k] != '0':
                            new_string = string[:j] + '0' + string[j+1:k] + '0' + string[k+1:]
                            new_output.add(new_string)
        output |= new_output

    output.discard(polarization_string) # remove the original string from the output
    return sorted(output) # return a sorted list of polarization strings.


In [ ]:
photons_2 = get_permutations([1,2]) 
photons_3 = get_permutations([1,2,4])
photons_4 = get_permutations([1, 2, 4, 7]) 
photons_5 = get_permutations([1, 2, 4, 7, 12]) 

photons = [photons_2, photons_3, photons_4, photons_5]

dictionaries = []

for i, photon_set in enumerate(photons, 2):
    dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
    flipped_list = []
    for j, sub_list in enumerate(photon_set):
        elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
        dictionary["Encoded Words"].append(elements)
        flipped = flip_polarizations(elements)
        flipped_list.append(flipped)
    dictionary[f"Received Loss Errors"] = flipped_list
    
    dictionaries.append(dictionary)


for i, dictionary in enumerate(dictionaries):
    print(f"Dictionary of {i+2} photons: {dictionary}")


In [ ]:


# # Define table headers
# headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# # Create table object
# table = PrettyTable(headers)

# # Add rows to table
# for d in dictionaries:
#     # Get photons and sums values from dictionary
#     photons = d["Sent Digits"]
#     encoded_words = d["Encoded Words"]
#     loss_errors = d["Received Loss Errors"]
#     #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
#     # Add rows to the table
#     for i in range(len(photons)):
#         row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
#         table.add_row(row)

# # Print table
# print(table)
# print(table[0:2])
# # print(table[2:8])
# # print(table[8:32])
# # print(table[32:1000])

# display several tables of the results as several image that corresponds to photons_2, photons_3, photons_4, photons_5 separately.
# rows = []
# for d in dictionaries:
#     # Get photons and sums values from dictionary
#     photons = d["Sent Digits"]
#     encoded_words = d["Encoded Words"]
#     loss_errors = d["Received Loss Errors"]
#     #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
#     # Add rows to the table
#     for i in range(len(photons)):
#         row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
#         rows.append(row)

# # Print table
# print(tabulate(rows, headers=headers, tablefmt="grid"))
# #print(tabulate(rows, headers=headers, tablefmt="grid", colwidths=[None, 20, None]))




# Define table headers
headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# Create table object
table = PrettyTable(headers)

# Set the maximum width for each column
table.max_width["Sent Digits"] = 15
table.max_width["Encoded Words"] = 30
table.max_width["Received Loss Errors"] = 30

# Add rows to table
for d in dictionaries:
    # Get photons and sums values from dictionary
    photons = d["Sent Digits"]
    encoded_words = d["Encoded Words"]
    loss_errors = d["Received Loss Errors"]
    #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
    # Add rows to the table
    for i in range(len(photons)):
        row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
        table.add_row(row)

# Print table
print(table)


# Correct the Error

In [ ]:
# define a function that can check if the input codeword corresponds to dictionary "Encoded Words" or "Received Loss Errors".
#If it belongs to the dictionary "Encoded Words", print("Your reveieved code word"(the input)" is correct, which corresponds to the sent digit "()" from the dictionary "Sent Digits").
#If it cannot be found in the dictionary "Encoded Words", search the codeword from the dictionary "Received Loss Errors".
# If it belonegs to the dictionary "Received Loss Errors", search it belongs to which row and find the correpsonging codeword from the dictionary "Encoded Words" ,
# then print("YOur received code word (the input) is incorrect, it should be the codedword (the cooresponding codewrod from Encoded Words) from the "Encoded Words",  which corresponds to the sent digit '(sent digit)' from the dictionary "Sent Digits"))
# For example, if the input is 'H0V000H', it belongs to the dictionary "Encoded Wrods", so it is correct. 
# Then it should print("Your reveieved code word  'H0V000H' is correct, which corresponds to the sent digit '(2,4,1)' from the dictionary "Sent Digits")
# Another example, if the input is '000H000V000000', it cannot be found in the dictionary "Encoded Words", then it should search the dictionary "Received Loss Errors".
# Then it should print("Your reveieved code word  '000H000V000000' is incorrect, it should be the codedword 'HH0H000V000000' from the "Encoded Words", which corresponds to the sent digit '(1,2,4,7))' from the dictionary "Sent Digits").

In [ ]:

def get_permutations(numbers):
    return list(permutations(numbers))

def convert_to_0s(photon):
    digit_to_0s = [(i-1)*"0" for i in photon]
    return digit_to_0s
    
def AddPolarization(digit_to_0s):
    result = ['']
    prev_non_empty = None
    
    for i, elem in enumerate(digit_to_0s):
        if i == 0:
            prev_non_empty = elem
            result.append(elem)
            continue
        
        if elem.count('0') > prev_non_empty.count('0'):
            result[-1] = 'H' + result[-1]
        elif elem.count('0') < prev_non_empty.count('0'):
            result[-1] = 'V' + result[-1]
        
        result.append(elem)
        prev_non_empty = elem
    
    if len(result[0]) < len(result[-1]):
        result[-1] = 'V' + result[-1][1:] + '0'
    elif len(result[0]) > len(result[-1]):
        result[-1] = 'H' + result[-1][1:] + '0'
    elif len(digit_to_0s) == 2:
        result[-1] = 'H'
    
    if len(digit_to_0s) > 1:
        last_digit_zeros = digit_to_0s[-1].count('0')
        second_last_digit_zeros = digit_to_0s[-2].count('0')
        if last_digit_zeros == 0:
            if second_last_digit_zeros > 0:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] += 'H'
        elif result[-1][-1] == '1':
            if last_digit_zeros > second_last_digit_zeros:
                result[-1] = result[-1][:-1] + 'V'
            elif second_last_digit_zeros > last_digit_zeros:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] = result[-1][:-1] + 'V' if result[-2][-1] == 'H' else result[-1][:-1] + 'H'
    
    return ''.join(result)      # Join the result list into a single string

def convert_to_one_string(numList):         # [1,2,3]
      s = map(str, numList)   # ['1','2','3']
      s = ''.join(s)          # '123'
      s = f'{s}'
      #s = int(s)              # 123
      return s

def flip_polarizations(polarization_string):
    output = {polarization_string}
    for i in range(1, len(polarization_string) + 1): # loop through all possible flip counts
        for j in range(len(polarization_string)): # loop through each polarization
            if polarization_string[j] != '0':
                output |= {polarization_string[:j] + '0' + polarization_string[j+1:]}
                for k in range(j+1, len(polarization_string)):
                    if polarization_string[k] != '0':
                        new_string = polarization_string[:j] + '0' + polarization_string[j+1:k] + '0' + polarization_string[k+1:]
                        output.add(new_string)
        if i == len(polarization_string): # stop flipping if there are no more polarizations to flip
            break
        new_output = set()
        for string in output:
            for j in range(len(polarization_string)):
                if string[j] != '0':
                    for k in range(j+1, len(polarization_string)):
                        if string[k] != '0':
                            new_string = string[:j] + '0' + string[j+1:k] + '0' + string[k+1:]
                            new_output.add(new_string)
        output |= new_output

    output.discard(polarization_string) # remove the original string from the output
    return sorted(output) # return a sorted list of polarization strings.


photons_2 = get_permutations([1,2]) 
photons_3 = get_permutations([1,2,4])
photons_4 = get_permutations([1, 2, 4, 7]) 
photons_5 = get_permutations([1, 2, 4, 7, 12]) 

photons = [photons_2, photons_3, photons_4, photons_5]

dictionaries = []

for i, photon_set in enumerate(photons, 2):
    dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
    flipped_list = []
    for j, sub_list in enumerate(photon_set):
        elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
        dictionary["Encoded Words"].append(elements)
        flipped = flip_polarizations(elements)
        flipped_list.append(flipped)
    dictionary[f"Received Loss Errors"] = flipped_list
    
    dictionaries.append(dictionary)


# for i, dictionary in enumerate(dictionaries):
#     print(f"Dictionary of {i+2} photons: {dictionary}")


# Define table headers
headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# Create table object
table = PrettyTable(headers)

# Set the maximum width for each column
table.max_width["Sent Digits"] = 15
table.max_width["Encoded Words"] = 30
table.max_width["Received Loss Errors"] = 30

# Add rows to table
for d in dictionaries:
    # Get photons and sums values from dictionary
    photons = d["Sent Digits"]
    encoded_words = d["Encoded Words"]
    loss_errors = d["Received Loss Errors"]
    #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
    # Add rows to the table
    for i in range(len(photons)):
        row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
        table.add_row(row)


def check_codeword(codeword, table):
    for row in table._rows:
        sent_digit = row[0]
        encoded_word = row[1]
        received_loss_error = row[2]

        if codeword == received_loss_error:
            print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return

    print("The provided code word is not found in the 'Received Loss Errors' column in the table.")

print(table)


print("Recevied Loss Error")
check_codeword("0V0", table) # I expect it to print(f"Your received code word '{0V0}' is incorrect, it should be the coded word '{HV0}' from the 'Encoded Words', which corresponds to the sent digit {(1,2)} from the dictionary 'Sent Digits'.")
check_codeword("00H", table) # I expect it to print(f"Your received code word '{0HH}' is incorrect, it should be the coded word '{V0H}' from the 'Encoded Words', which corresponds to the sent digit {(2,1)} from the dictionary 'Sent Digits'.")
check_codeword("0V000V0", table) # I expect it to print(f"Your received code word '{0V000V0}' is incorrect, it should be the coded word '{HV000V0}' from the 'Encoded Words', which corresponds to the sent digit {1,4,2)} from the dictionary 'Sent Digits'.")

In [ ]:
def find_codeword(input_codeword):
    encoded_words = {
        "H0V000H": (2, 4, 1),
        # Add other encoded words here
    }

    received_loss_errors = {
        "000H000V000000": "HH0H000V000000",
        # Add other received loss errors here
    }

    sent_digits = {
        "H0V000H": (2, 4, 1),
        "HH0H000V000000": (1, 2, 4, 7),
        # Add other sent digits here
    }

    if input_codeword in encoded_words:
        print(f"Your received code word '{input_codeword}' is correct, which corresponds to the sent digit {encoded_words[input_codeword]} from the dictionary 'Sent Digits'")
    elif input_codeword in received_loss_errors:
        corrected_codeword = received_loss_errors[input_codeword]
        print(f"Your received code word '{input_codeword}' is incorrect, it should be the coded word '{corrected_codeword}' from the 'Encoded Words', which corresponds to the sent digit {sent_digits[corrected_codeword]} from the dictionary 'Sent Digits'")
    else:
        print("Input code word not found in either 'Encoded Words' or 'Received Loss Errors' dictionaries")


def check_codeword(codeword):
    encoded_words = {"H0V000H": (2, 4, 1),}# Add more encoded words here
    received_loss_errors = { "000H000V000000": "HH0H000V000000",
    # Add more received loss errors here
    }

    sent_digits = {
        (2, 4, 1): "H0V000H",
        (1, 2, 4, 7): "HH0H000V000000",
        # Add more sent digits here
        }

    if codeword in encoded_words:
        sent_digit = encoded_words[codeword]
        print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
    elif codeword in received_loss_errors:
        corrected_codeword = received_loss_errors[codeword]
        sent_digit = encoded_words[corrected_codeword]
        print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{corrected_codeword}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
    else:
        print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' dictionaries.")



In [ ]:
print(find_codeword("H0V000H"))
print(find_codeword("000H000V000000"))

In [ ]:
print(check_codeword("H0V000H"))
print(check_codeword("000H000V000000"))

In [ ]:

from prettytable import PrettyTable


def check_codeword_prettytable(codeword, table):
    encoded_word_column = headers.index("Encoded Words")
    sent_digit_column = headers.index("Sent Digits")
    received_loss_error_column = headers.index("Received Loss Errors")

    found = False
    for row in table:
        if codeword == row[encoded_word_column]:
            sent_digit = row[sent_digit_column]
            print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            found = True
            break
        elif codeword == row[received_loss_error_column]:
            corrected_codeword = row[encoded_word_column]
            sent_digit = row[sent_digit_column]
            print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{corrected_codeword}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            found = True
            break

    if not found:
        print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns of the table.")

# Create the PrettyTable object and add rows
table = PrettyTable(headers)
table.add_row([(2, 4, 1), "H0V000H", "000H000V000000"])
# Add more rows as needed

check_codeword_prettytable("HV0", table)


In [ ]:
# Example usage:
# check_codeword_prettytable("H0V000H", table)
#check_codeword_prettytable("000H000V000000", table)


In [ ]:
from prettytable import PrettyTable

photons_2 = get_permutations([1,2]) 
photons_3 = get_permutations([1,2,4])
photons_4 = get_permutations([1, 2, 4, 7]) 
photons_5 = get_permutations([1, 2, 4, 7, 12]) 

photons = [photons_2, photons_3, photons_4, photons_5]

dictionaries = []

# for i, photon_set in enumerate(photons, 2):
#     dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
#     flipped_list = []
#     for j, sub_list in enumerate(photon_set):
#         elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
#         dictionary["Encoded Words"].append(elements)
#         flipped = flip_polarizations(elements)
#         flipped_list.append(flipped)
#     dictionary[f"Received Loss Errors"] = flipped_list
    
#     dictionaries.append(dictionary)

for i, photon_set in enumerate(photons, 2):
    dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
    flipped_list = []
    for j, sub_list in enumerate(photon_set):
        elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
        dictionary["Encoded Words"].append(elements)
        flipped = flip_polarizations(elements)
        # Join the flipped list into a single string
        flipped_string = ''.join(flipped)
        flipped_list.append(flipped_string)
    dictionary[f"Received Loss Errors"] = flipped_list
    
    dictionaries.append(dictionary)


for i, dictionary in enumerate(dictionaries):
    print(f"Dictionary of {i+2} photons: {dictionary}")


# Define table headers
headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# Create table object
table = PrettyTable(headers)

# Set the maximum width for each column
table.max_width["Sent Digits"] = 15
table.max_width["Encoded Words"] = 30
table.max_width["Received Loss Errors"] = 30

# Add rows to table
for d in dictionaries:
    # Get photons and sums values from dictionary
    photons = d["Sent Digits"]
    encoded_words = d["Encoded Words"]
    loss_errors = d["Received Loss Errors"]
    #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
    # Add rows to the table
    for i in range(len(photons)):
        row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
        table.add_row(row)

def check_codeword(codeword, table):
    if all(c == '0' for c in codeword):
        print("I cannot determine the correct code word. Please check the table.")
        return

    for row in table._rows:
        sent_digit = row[0]
        encoded_word = row[1]
        received_loss_error = row[2]

        # Split the received_loss_error into individual characters
        received_loss_error_split = ''.join([c for c in received_loss_error])

        if codeword == encoded_word:
            print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return
        elif received_loss_error and codeword == received_loss_error_split:
            print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return

    print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.")


# def check_codeword(codeword, table):
#     if all(c == '0' for c in codeword):
#         print("I cannot determine the correct code word. Please check the table.")
#         return

#     for row in table._rows:
#         sent_digit = row[0]
#         encoded_word = row[1]
#         received_loss_error = row[2]

#         # # Split the received_loss_error into individual characters
#         # received_loss_error_split = ''.join([c for c in received_loss_error])

#         # if codeword == encoded_word:
#         #     print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
#         #     return
#         if codeword == received_loss_error:
#             print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
#             return

#     print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.")

# def check_codeword(codeword, table):
#     for row in table._rows:
#         sent_digit = row[0]
#         encoded_word = row[1]
#         received_loss_error = row[2]

#         if codeword == received_loss_error:
#             print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
#             return

#     print("The provided code word is not found in the 'Received Loss Errors' column in the table.")




# # Example usage:
# headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"]
# table = PrettyTable(headers)
# table.add_row(["(2, 4, 1)", "H0V000H", ""])
# table.add_row(["(1, 2, 4, 7)", "HH0H000V000000", "000H000V000000"])


# add one more line to the function, if all digits are 0, then print(" I cannot determine the correct code word. Please check the table. ")



print("Recevied Loss Error")
check_codeword("HV0", table) # I expect it to print(f"Your received code word '{0V0}' is incorrect, it should be the coded word '{HV0}' from the 'Encoded Words', which corresponds to the sent digit {(1,2)} from the dictionary 'Sent Digits'.")
check_codeword("00H", table) # I expect it to print(f"Your received code word '{0HH}' is incorrect, it should be the coded word '{V0H}' from the 'Encoded Words', which corresponds to the sent digit {(2,1)} from the dictionary 'Sent Digits'.")
check_codeword("0V000V0", table) # I expect it to print(f"Your received code word '{0V000V0}' is incorrect, it should be the coded word '{HV000V0}' from the 'Encoded Words', which corresponds to the sent digit {1,4,2)} from the dictionary 'Sent Digits'.")
check_codeword("000H000V000000", table) # I expect it to print(f"Your received code word '{0V000V0}' is incorrect, it should be the coded word '{HH0H000V000000}' from the 'Encoded Words', which corresponds to the sent digit {1,2, 4,7)} from the dictionary 'Sent Digits'.")


In [ ]:
print(table)

In [ ]:
check_codeword("0V0", table)
check_codeword("00H", table)

In [ ]:
def correcterror(codeword):
    sent_digits = {"(1,2,3)": "H0H", "(2,4,1)": "H0V000H", "(3,1,4,1)": "HV00H0V000H"}
    encoded_words = {"H0H": "000000", "H0V000H": "000H000V000000", "HV00H0V000H": "H00V0H0V0H00"}
    received_loss_errors = [["H0H000H", "H00H00H", "H0HHH0H", "H0H0HHH"], ["H0V000H", "H00V00H", "H0VVV0H", "H0V0VVV"], ["HV00H0V000H", "HVV0H0V000H", "HV00VVV000H", "HV0HH0V00VH"]]
    
    if codeword in encoded_words.values():
        sent_digit = [k for k, v in sent_digits.items() if v == codeword][0]
        print("Your received code word '{}' is correct, which corresponds to the sent digit '{}' from the dictionary 'Sent Digits'.".format(codeword, sent_digit))
    else:
        for i, row in enumerate(received_loss_errors):
            if codeword in row:
                correct_codeword = encoded_words[sent_digits.keys()[sent_digits.values().index(received_loss_errors[i][0])]]
                sent_digit = sent_digits.keys()[sent_digits.values().index(correct_codeword)]
                print("Your received code word '{}' is incorrect, it should be the codedword '{}' from the 'Encoded Words', which corresponds to the sent digit '{}' from the dictionary 'Sent Digits'.".format(codeword, correct_codeword, sent_digit))
                break
        else:
            print("The provided codeword is not valid.")


In [ ]:
# define a function multple from 1 to n and then add 1 to n 

def multipleandadd(n):
    return (n*(n+1))/2


In [ ]:
# import csv

# # Define output file name
# output_file = "Downloads/BPPM_Errors.csv"

# # Write table to CSV file
# with open(output_file, 'w', newline='') as f:
#     writer = csv.writer(f)
#     # Write headers
#     writer.writerow(headers)
#     # Write rows
#     for row in table:
#         writer.writerow(row)


In [ ]:
# import csv

# # Define output file name
# output_file = "Downloads/BPPM_Errors.csv"

# # Write table rows to CSV file
# with open(output_file, 'w', newline='') as f:
#     writer = csv.writer(f)
#     # Write headers
#     writer.writerow(headers)
#     # Write rows
#     writer.writerows(rows)


In [ ]:
# import tabulate

# # Set column widths
# column_widths = [8, 40, 20]

# # Create table
# table = tabulate.PrettyTable(headers, hrules=tabulate.ALL, colalign=["center"] * len(headers), colwidths=column_widths)

# # Add rows to table
# for d in dictionaries:
#     # Get photons and sums values from dictionary
#     photons = d["Sent Digits"]
#     encoded_words = d["Encoded Words"]
#     loss_errors = d["Received Loss Errors"]

#     # Add rows to the table
#     for i in range(len(photons)):
#         row = [photons[i], encoded_words[i], loss_errors[i]]
#         table.add_row(row)

# # Print table
# print(table)


In [ ]:
# # Set width for text wrapping
# col_width = 30

# # Create list of rows
# rows = []
# for d in dictionaries:
#     # Get photons and sums values from dictionary
#     photons = d["Sent Digits"]
#     encoded_words = d["Encoded Words"]
#     loss_errors = d["Received Loss Errors"]

#     # Add rows to the list
#     for i in range(len(photons)):
#         # Wrap text in the Encoded Words column
#         wrapped_encoded_words = '\n'.join(textwrap.wrap(encoded_words[i], col_width))

#         row = [photons[i], wrapped_encoded_words, loss_errors[i]]
#         rows.append(row)

# # Print table
# print(table(rows, headers=headers, tablefmt="grid"))